In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import opendatasets as od
pd.options.plotting.backend = "plotly"


## Download & Import the dataset

* Requires a Kaggle API Key

In [13]:
# od.download('https://www.kaggle.com/datasets/lamiatabassum/top-50-us-tech-companies-2022-2023-dataset')

In [63]:
df = pd.read_csv('top-50-us-tech-companies-2022-2023-dataset/Top 50 US Tech Companies 2022 - 2023.csv')

## What does the dataset contain?

We can see we have the top 50 US Tech companies and the dataset also provides us with some basic figures for said companies.

Descriptive: Company Name, Industry, Sector, HQ State, Stock Name, Founding Year

Size of the company/Financials: Annual Revenue 2022-2023 (USD in Billions), Market Cap (USD in Trillions), Annual Income Tax in 2022-2023 (USD in Billions), Employee Size

In [64]:
df.shape

(50, 10)

In [65]:
df.head()

,Company Name,Industry,Sector,HQ State,Founding Year,Annual Revenue 2022-2023 (USD in Billions),Market Cap (USD in Trillions),Stock Name,Annual Income Tax in 2022-2023 (USD in Billions),Employee Size
0,Apple Inc.,Technology,Consumer Electronics,California,1976,387.53,2.520,AAPL,18.314,164000
1,Microsoft Corporation,Technology,Software Infrastructure,Washington,1975,204.09,2.037,MSFT,15.139,221000
2,Alphabet (Google),Technology,Software Infrastructure,California,1998,282.83,1.350,GOOG,11.356,190234
3,Amazon,Technology,Software Application,Washington,1994,513.98,1.030,AMZN,-3.217,1541000
4,NVIDIA Corporation,Technology,Semiconductors,California,1993,26.97,0.653,NVDA,0.189,22473


## Basic Statistics

In [66]:
df.describe()

,Founding Year,Annual Revenue 2022-2023 (USD in Billions),Market Cap (USD in Trillions),Annual Income Tax in 2022-2023 (USD in Billions),Employee Size
count,50.000000,50.00000,50.000000,50.000000,5.000000e+01
mean,1984.140000,51.20440,0.252160,1.386780,8.324962e+04
std,24.988985,97.41288,0.490377,3.687916,2.205869e+05
min,1890.000000,2.06000,0.028000,-3.217000,2.993000e+03
25%,1977.250000,7.65250,0.051250,0.098750,1.415000e+04
50%,1988.500000,17.66500,0.082500,0.280500,2.472500e+04
75%,1999.750000,40.81500,0.160250,0.945000,7.015575e+04
max,2012.000000,513.98000,2.520000,18.314000,1.541000e+06


In [67]:
df.describe(include=['O'])

,Company Name,Industry,Sector,HQ State,Stock Name
count,50,50,50,50,50
unique,50,1,8,13,50
top,Micron Technology,Technology,Software Application,California,KLAC
freq,1,50,15,33,1


In [69]:
## Are there any null values
df.isnull().sum()

Company Name                                        0
Industry                                            0
Sector                                              0
HQ State                                            0
Founding Year                                       0
Annual Revenue 2022-2023 (USD in Billions)          0
Market Cap (USD in Trillions)                       0
Stock Name                                          0
Annual Income Tax in 2022-2023 (USD in Billions)    0
Employee Size                                       0
dtype: int64

## Initial EDA

In [72]:
Sector = df['Sector'].value_counts()

fig = px.pie(names = Sector.index, values = Sector.values)
fig.update_traces(textinfo='label+percent+value', title= 'Sector')
fig.show()

In [75]:
state_code = {'Alabama': 'AL',
        'Alaska': 'AK',
        'Arizona': 'AZ',
        'Arkansas': 'AR',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'District of Columbia': 'DC',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Iowa': 'IA',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Maine': 'ME',
        'Maryland': 'MD',
        'Massachusetts': 'MA',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Mississippi': 'MS',
        'Missouri': 'MO',
        'Montana': 'MT',
        'Nebraska': 'NE',
        'Nevada': 'NV',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'New York': 'NY',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Vermont': 'VT',
        'Virginia': 'VA',
        'Washington': 'WA',
        'West Virginia': 'WV',
        'Wisconsin': 'WI',
        'Wyoming': 'WY'}

In [78]:
df['HQ State Code'] = df['HQ State'].map(state_code)

In [83]:
volume_per_area = (df[['HQ State Code','HQ State','Company Name']]
                   .groupby(['HQ State Code','HQ State'],as_index=False)
                   .count()
                   .rename(columns={'Company Name':'Volume'})
                  )

In [113]:
fig = px.choropleth(volume_per_area,
                    locations='HQ State Code',
                    color='Volume',
                    color_continuous_scale='spectral_r',
                    hover_name='HQ State',
                    locationmode='USA-states',
                    labels={'HQ State':'Volume'},
                    scope='usa'
                   )

fig.add_scattergeo(
    locations=volume_per_area['HQ State Code'],
    locationmode='USA-states',
    text=volume_per_area['HQ State Code'],
    mode='text'
)

fig.update_layout(title= {'text':'Volume of Companies per State',
                          'xanchor':'center',
                          'yanchor':'top',
                          'x':0.5})
fig.show()

## Sector Analysis

In [153]:
cols = ['Annual Revenue 2022-2023 (USD in Billions)','Annual Income Tax in 2022-2023 (USD in Billions)',
        'Employee Size','Market Cap (USD in Trillions)']

for col in cols:
    new_df = pd.merge((df[['Sector',
                           col]]
                       .groupby('Sector')
                       .sum()
                      ),
                      (df[['Sector',
                           col]]
                       .groupby('Sector')
                       .mean()
                      ),
                      left_index=True,
                      right_index=True,
                     suffixes=(' Total',' Mean'))
    fig = new_df.plot(kind='bar')

    fig.update_layout(title= {'text':f'{col} per Sector'})
    fig.update_layout(barmode='group')
    fig.show()
    

## State Analysis

In [154]:
for col in cols:
    new_df = pd.merge((df[['HQ State',
                           col]]
                       .groupby('HQ State')
                       .sum()
                      ),
                      (df[['HQ State',
                           col]]
                       .groupby('HQ State')
                       .mean()
                      ),
                      left_index=True,
                      right_index=True,
                     suffixes=(' Total',' Mean'))
    fig = new_df.plot(kind='bar')

    fig.update_layout(title= {'text':f'{col} per State'})
    fig.update_layout(barmode='group')
    fig.show()
    

## Correlation

In [156]:
corr

,Annual Revenue 2022-2023 (USD in Billions),Annual Income Tax in 2022-2023 (USD in Billions),Employee Size,Market Cap (USD in Trillions)
Annual Revenue 2022-2023 (USD in Billions),1.000000,0.545949,0.795009,0.806985
Annual Income Tax in 2022-2023 (USD in Billions),0.545949,1.000000,-0.019060,0.878740
Employee Size,0.795009,-0.019060,1.000000,0.376516
Market Cap (USD in Trillions),0.806985,0.878740,0.376516,1.000000


In [159]:
corr

,Annual Revenue 2022-2023 (USD in Billions),Annual Income Tax in 2022-2023 (USD in Billions),Employee Size,Market Cap (USD in Trillions)
Annual Revenue 2022-2023 (USD in Billions),1.000,0.546,0.795,0.807
Annual Income Tax in 2022-2023 (USD in Billions),0.546,1.000,-0.019,0.879
Employee Size,0.795,-0.019,1.000,0.377
Market Cap (USD in Trillions),0.807,0.879,0.377,1.000


In [160]:
import plotly.figure_factory as ff

corr = df[cols].corr().round(3)
fig = ff.create_annotated_heatmap(z=corr.to_numpy(), 
                                  x=corr.columns.tolist(),
                                  y=corr.columns.tolist().reverse(),
                                  colorscale=px.colors.diverging.RdBu,
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )
fig.show()